In [34]:
import numpy as np

training_data_length = 1000
training_data = np.zeros((training_data_length, 4), dtype=int)

for i in range(training_data_length):
    pattern = np.array([1, 1, 0, 0])
    np.random.shuffle(pattern)
    training_data[i] = pattern

def getCorrectResults(data):
    return np.array([1 if list(pattern) == [1, 0, 0, 1] else 0 for pattern in data])

correct_results_training_data = getCorrectResults(training_data)

In [35]:
import tensorflow.keras as keras


NN = keras.Sequential() 
NN.add(keras.layers.Dense(1, activation="sigmoid", input_dim=4))

NN.compile(loss="binary_crossentropy", optimizer="Adam")

NN.fit(training_data, correct_results_training_data, epochs=100)

NN.summary()

weights = NN.get_weights()

# Print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i}:")
    for j, weight in enumerate(layer_weights):
        print(f"Weight {j}: {weight}")


Epoch 1/100


C:\Users\thoms\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - loss: 0.7424 
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - loss: 0.7202
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 0.6837
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - loss: 0.6481
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - loss: 0.6312
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 0.6473
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 0.6013
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 0.5857
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - loss: 0.5591
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - loss: 0.5397
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 0.5204
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 0.5062
Epoch 13/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 0.5224
Epoch 14/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - loss: 0.4737
Epoch 15/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step 

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17 (72.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12 (52.00 B)

Layer 0:
Weight 0: [0.9467115]
Weight 1: [-2.810489]
Weight 2: [-1.4100311]
Weight 3: [1.0788964]
Layer 1:
Weight 0: -1.2742069959640503


In [36]:
results_on_training_data = NN.predict(training_data)

for i in range(training_data_length):
    print(training_data[i], " -> ", bool(np.round(results_on_training_data[i])))
    print("correct result : ", bool(correct_results_training_data[i]))
    print('\n')

print((np.round(results_on_training_data)) == correct_results_training_data)



32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
[1 0 0 1]  ->  True
correct result :  True


[0 0 1 1]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[1 0 0 1]  ->  True
correct result :  True


[0 0 1 1]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[0 0 1 1]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[0 0 1 1]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[0 0 1 1]  ->  False
correct result :  False


[1 0 0 1]  ->  True
correct result :  True


[1 0 1 0]  ->  False
corr

In [37]:
test_data = np.copy(training_data)
np.random.shuffle(test_data)

results_on_test_data = NN.predict(test_data)
correct_results_test_data = getCorrectResults(test_data)

all_correct = True

for i in range(training_data_length):
    print(test_data[i], " -> ", bool(np.round(results_on_test_data[i])))
    print("correct result : ", bool(correct_results_test_data[i]))
    print('\n')
    if all_correct:
        all_correct = bool(np.round(results_on_test_data[i])) == bool(correct_results_test_data[i])

print("all test data correct: ", all_correct)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
[0 1 0 1]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[1 0 1 0]  ->  False
correct result :  False


[1 0 0 1]  ->  True
correct result :  True


[1 0 0 1]  ->  True
correct result :  True


[1 1 0 0]  ->  False
correct result :  False


[1 0 0 1]  ->  True
correct result :  True


[0 0 1 1]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[0 0 1 1]  ->  False
correct result :  False


[0 1 1 0]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[1 1 0 0]  ->  False
correct result :  False


[0 1 0 1]  ->  False
correct result :  False


[1 1 0 0]  ->  False
corr